# Getting Started

## Wrapping a PyTorch model
Create a simple PyTorch model.

In [1]:
import torch
import numpy as np
import torch.nn as nn

class MLP(nn.Module):
    """
    Simple MLP.
    """
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.mlp(x)

class SimpleModel(nn.Module):
    """
    Simple model class.
    """
    def __init__(self, input_dim, output_dim, hidden_dim = 64):
        super(SimpleModel, self).__init__()

        self.mlp = MLP(input_dim, output_dim, hidden_dim)

    def forward(self, x):
        x = self.mlp(x)
        return x

Train the model on some data.

$$
y = x_0^2 +3 \sin(x_4)-2
$$

In [2]:
# Make the dataset 
x = np.array([np.random.uniform(0, 1, 10_000) for _ in range(5)]).T  
y = x[:, 0]**2 + 3*np.sin(x[:, 4]) - 4
noise = np.array([np.random.normal(0, 0.05*np.std(y)) for _ in range(len(y))])
y = y + noise 

In [3]:
# Set up training

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def train_model(model, dataloader, opt, criterion, epochs = 100):
    """
    Train a model for the specified number of epochs.
    
    Args:
        model: PyTorch model to train
        dataloader: DataLoader for training data
        opt: Optimizer
        criterion: Loss function
        epochs: Number of training epochs
        
    Returns:
        tuple: (trained_model, loss_tracker)
    """
    loss_tracker = []
    for epoch in range(epochs):
        epoch_loss = 0.0
        
        for batch_x, batch_y in dataloader:
            # Forward pass
            pred = model(batch_x)
            loss = criterion(pred, batch_y)
            
            # Backward pass
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            epoch_loss += loss.item()
        
        loss_tracker.append(epoch_loss)
        if (epoch + 1) % 5 == 0:
            avg_loss = epoch_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{epochs}], Avg Loss: {avg_loss:.6f}')
    return model, loss_tracker

# Instantiate the model
model = SimpleModel(input_dim=x.shape[1], output_dim=1)

# Set up training
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=0.001)
X_train, _, y_train, _ = train_test_split(x, y.reshape(-1,1), test_size=0.2, random_state=290402)

# Set up dataset
dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
# Train the model and save the weights

model, losses = train_model(model, dataloader, opt, criterion, 20)
torch.save(model.state_dict(), 'model_weights.pth')

Epoch [5/20], Avg Loss: 0.091779
Epoch [10/20], Avg Loss: 0.065450
Epoch [15/20], Avg Loss: 0.048581
Epoch [20/20], Avg Loss: 0.038570


Wrap the mlp layer in the trained model with MLP_SR.

In [5]:
from symtorch.mlp_sr import MLP_SR
model.mlp = MLP_SR(model.mlp, mlp_name = 'Sequential')

/Users/liz/PhD/InterpretSR_project/interpretsr_venv/lib/python3.11/site-packages/juliacall/__init__.py:61: UserWarning: torch was imported before juliacall. This may cause a segfault. To avoid this, import juliacall before importing torch. For updates, see https://github.com/pytorch/pytorch/issues/78829.
  warnings.warn(


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


## Interpret the MLP

In this example, we pass extra parameters into the `.interpret` method (complexity of operators/constants and parsimony, which is a penalisation of complexity).\
To see all the possible parameters, please see the `PySRRegressor` class from [PySR](https://ai.damtp.cam.ac.uk/pysr/api/).

In this example, we turn verbosity off because we are in a Jupyter notebook. For best performance, run in IPython, as you can terminate the SR any time.

In [6]:
model.mlp.interpret(torch.FloatTensor(X_train), 
                       niterations = 500, # Should set to higher
                       verbosity=0, 
                       complexity_of_operators = {"sin":3, "exp":3}, 
                       complexity_of_constants = 2,
                       parsimony = 0.1)

🛠️ Running SR on output dimension 0 of 1


/Users/liz/PhD/InterpretSR_project/interpretsr_venv/lib/python3.11/site-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


💡Best equation for output 0 found to be ((x0 * x0) * 0.9140018) + inv(((x4 * (x4 + 0.80192065)) + 1.2257804) * -0.2080118).
❤️ SR on Sequential complete.


{0: PySRRegressor.equations_ = [
 	    pick     score                                           equation  \
 	0         0.000000                                                 x4   
 	1         2.694948                                          -2.320417   
 	2         0.421408                                     x4 + -2.826031   
 	3         0.479556                             x4 + (x4 + -3.3315902)   
 	4         0.137519                      (x4 + -1.4832371) * 2.3734488   
 	5         1.247076                      ((x0 + x4) + x4) + -3.8293786   
 	6         0.775753                 (x0 + (x4 * 2.382997)) + -4.023018   
 	7         0.231667        ((x4 * 2.3833776) + -3.8567781) + (x0 * x0)   
 	8         0.059380          (sin(x4) * 2.7949803) + (x0 + -4.1155796)   
 	9         0.496750     (x0 * x0) + ((sin(x4) * 2.795451) + -3.949366)   
 	10        0.039731  (inv(x4 + ((x4 * x4) + 1.3869555)) * -5.523348...   
 	11        0.211374  ((x0 * 0.9184323) * x0) + ((sin(x4) + -1.4034

See the full Pareto front of equations. The best equation is chosen as a balance of accuracy and complexity.\
Outputs from *PySR* are saved in `SR_output/MLP_name`.

In [7]:
print(model.mlp.pysr_regressor)

{0: PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                                 x4   
	1         2.694948                                          -2.320417   
	2         0.421408                                     x4 + -2.826031   
	3         0.479556                             x4 + (x4 + -3.3315902)   
	4         0.137519                      (x4 + -1.4832371) * 2.3734488   
	5         1.247076                      ((x0 + x4) + x4) + -3.8293786   
	6         0.775753                 (x0 + (x4 * 2.382997)) + -4.023018   
	7         0.231667        ((x4 * 2.3833776) + -3.8567781) + (x0 * x0)   
	8         0.059380          (sin(x4) * 2.7949803) + (x0 + -4.1155796)   
	9         0.496750     (x0 * x0) + ((sin(x4) * 2.795451) + -3.949366)   
	10        0.039731  (inv(x4 + ((x4 * x4) + 1.3869555)) * -5.523348...   
	11        0.211374  ((x0 * 0.9184323) * x0) + ((sin(x4) + -1.40345...   
	12  

## Switch to Using the Equation Instead in the Forwards Pass

You can choose the equation you want to switch to by choosing the desired complexity of equation. \
If left blank, then we choose the best equation chosen by *PySR*.

In [8]:
model.mlp.switch_to_equation(complexity=14) 

✅ Successfully switched Sequential to symbolic equations for all 1 dimensions:
   Dimension 0: (x0 * x0) + ((sin(x4) * 2.795451) + -3.949366)
   Variables: ['x0', 'x4']
🎯 All 1 output dimensions now using symbolic equations.


Now when running the forwards pass through the model, it uses the symbolic equation instead of the MLP. 

In [26]:
interpretable_outputs = model(torch.tensor(X_train, dtype=torch.float32))
interpretable_outputs

tensor([[-3.8636],
        [-3.4790],
        [-1.5592],
        ...,
        [-2.7508],
        [-2.6754],
        [-3.3049]])

## Switch to Using the MLP in the Forwards Pass

In [28]:
mlp_outputs = model.mlp.switch_to_mlp()

✅ Switched Sequential back to MLP


In [29]:
model.eval()
with torch.no_grad():
    model_outputs = model.mlp(torch.tensor(X_train, dtype=torch.float32))
model_outputs

tensor([[-3.8313],
        [-3.5179],
        [-1.5196],
        ...,
        [-2.7176],
        [-2.6945],
        [-3.3658]])

In [32]:
model_outputs.shape

torch.Size([8000, 1])

In [23]:
# Clean up 
import os
import shutil
if os.path.exists('SR_output'):
    shutil.rmtree('SR_output')
os.remove('model_weights.pth')